In [1]:
# import kagglehub

# # Download LFW dataset
# path = kagglehub.dataset_download("jessicali9530/lfw-dataset")

# print("Path to dataset files:", path)


In [ ]:
import shutil, random
from pathlib import Path

# ====== CONFIG ======
SRC = Path("/home/tuanh/projet/face_Recognition/image")
OUT = Path("/home/tuanh/projet/face_Recognition/dataset")

VAL_RATIO = 0.1
SEED = 42
EXTS = {".jpg", ".jpeg", ".png"}

MIN_IMGS = 2               # bỏ class < 2 ảnh (không thể split train/val)
MAX_IMGS = None            # ví dụ 30 nếu muốn cap mỗi class 30 ảnh
CLEAN_OUT = True           # xoá dataset cũ trước khi tạo lại

FORCE_VAL_EACH_CLASS = True  
# ====================

random.seed(SEED)

train_root = OUT / "train"
val_root   = OUT / "val"

def list_imgs(person_dir: Path):
    return sorted([f for f in person_dir.iterdir()
                   if f.is_file() and f.suffix.lower() in EXTS])

def clean_dirs():
    if CLEAN_OUT:
        shutil.rmtree(train_root, ignore_errors=True)
        shutil.rmtree(val_root, ignore_errors=True)
    train_root.mkdir(parents=True, exist_ok=True)
    val_root.mkdir(parents=True, exist_ok=True)

def copy_files(files, dst_dir: Path):
    dst_dir.mkdir(parents=True, exist_ok=True)
    for f in files:
        shutil.copy2(f, dst_dir / f.name)

clean_dirs()

people = [p for p in SRC.iterdir() if p.is_dir()]
print("Num identities (folders) in SRC:", len(people))

kept = 0
skipped = 0
train_imgs_total = 0
val_imgs_total = 0

train_classes = set()
val_classes = set()

for person_dir in sorted(people, key=lambda x: x.name):
    person = person_dir.name
    imgs = list_imgs(person_dir)
    n = len(imgs)

    if n < MIN_IMGS:
        skipped += 1
        continue

    # shuffle
    random.shuffle(imgs)

    # optional cap
    if MAX_IMGS is not None and n > MAX_IMGS:
        imgs = imgs[:MAX_IMGS]
        n = len(imgs)

    # ---- split ----
    if FORCE_VAL_EACH_CLASS:
        # đảm bảo: val >= 1 và train >= 1
        n_val = max(1, int(round(n * VAL_RATIO)))
        if n_val >= n:
            n_val = n - 1
        # vì MIN_IMGS>=2 nên n-1 >=1 ok
    else:
        # split "tự nhiên"
        n_val = int(n * VAL_RATIO)
        if n_val >= n:
            n_val = n - 1
        # có thể n_val = 0 -> class sẽ không có val

    val_imgs = imgs[:n_val]
    train_imgs = imgs[n_val:]

    # đảm bảo train luôn có ít nhất 1 ảnh (an toàn)
    if len(train_imgs) == 0:
        train_imgs = val_imgs[-1:]
        val_imgs = val_imgs[:-1]

    # copy
    if train_imgs:
        copy_files(train_imgs, train_root / person)
        train_classes.add(person)
        train_imgs_total += len(train_imgs)

    if val_imgs:
        copy_files(val_imgs, val_root / person)
        val_classes.add(person)
        val_imgs_total += len(val_imgs)

    kept += 1
    print(f"{person:25s} total={n:3d}  train={len(train_imgs):3d}  val={len(val_imgs):3d}")

print("\n==== DONE ====")
print("Kept identities :", kept)
print("Skipped (<MIN)  :", skipped)
print("Train images    :", train_imgs_total)
print("Val images      :", val_imgs_total)
total = train_imgs_total + val_imgs_total
print("Total images    :", total)
print("Val ratio real  :", (val_imgs_total / total) if total else 0.0)
print("Train dir:", train_root)
print("Val dir  :", val_root)

# ===== sanity check: train/val class mismatch =====
only_train = sorted(train_classes - val_classes)
only_val   = sorted(val_classes - train_classes)

print("\n==== CLASS CONSISTENCY CHECK ====")
print("Train classes:", len(train_classes))
print("Val classes  :", len(val_classes))

if only_train or only_val:
    print("⚠️ MISMATCH DETECTED!")
    if only_train:
        print("Classes only in TRAIN (first 20):", only_train[:20])
    if only_val:
        print("Classes only in VAL   (first 20):", only_val[:20])
else:
    print("✅ OK: Train/Val have identical class sets.")


Num identities (folders) in SRC: 5749
Aaron_Peirsol             total=  4  train=  3  val=  1
Aaron_Sorkin              total=  2  train=  1  val=  1
Abdel_Nasser_Assidi       total=  2  train=  1  val=  1
Abdoulaye_Wade            total=  4  train=  3  val=  1
Abdullah                  total=  4  train=  3  val=  1
Abdullah_Gul              total= 19  train= 17  val=  2
Abdullah_al-Attiyah       total=  3  train=  2  val=  1
Abdullatif_Sener          total=  2  train=  1  val=  1
Abel_Pacheco              total=  4  train=  3  val=  1
Abid_Hamid_Mahmud_Al-Tikriti total=  3  train=  2  val=  1
Adam_Sandler              total=  4  train=  3  val=  1
Adam_Scott                total=  2  train=  1  val=  1
Adel_Al-Jubeir            total=  3  train=  2  val=  1
Adolfo_Aguilar_Zinser     total=  3  train=  2  val=  1
Adolfo_Rodriguez_Saa      total=  2  train=  1  val=  1
Adrian_McPherson          total=  2  train=  1  val=  1
Adrian_Nastase            total=  2  train=  1  val=  1
Adrien_

In [ ]:

from pathlib import Path
import collections

root = Path("dataset/train")
cnt = []
for d in root.iterdir():
    if d.is_dir():
        cnt.append(len(list(d.glob("*.jpg"))))
c = collections.Counter(cnt)
print("num_classes:", len(cnt))
print("min/max:", min(cnt), max(cnt))
for k in sorted(c)[:10]:
    print(k, c[k])
PY


SyntaxError: invalid syntax (838413734.py, line 1)